In [1]:
import pandas as pd
import numpy as np
import re
import time

# df_source = pd.read_csv('../big_5_analysis.tsv', sep='\t')
df_source = pd.read_csv('../../Prompt-Personality/data/Essay/Essay_E_whole.tsv', sep='\t')
df_source.head()

,utterance,labels
0,"Well, right now I just woke up from a mid-day ...",0
1,"Well, here we go with the stream of consciousn...",0
2,An open keyboard and buttons to push. The thin...,0
3,I can't believe it! It's really happening! M...,1
4,"Well, here I go with the good old stream of co...",1


In [2]:
## get vad dict
VAD_Lexicons = pd.read_csv('../data/NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt', sep='\t')
VAD_dict = {}
for r in VAD_Lexicons.iterrows():
    VAD_dict[r[1]['Word']] = [r[1]['Valence'], r[1]['Arousal'], r[1]['Dominance']]

    
def get_VAD_tokenized_dict(i, VAD_dict):
    try:
        return VAD_dict[i]
    except:
        return [0.0, 0.0, 0.0]



In [3]:
import nltk
from nltk.corpus import stopwords
s_list = stopwords.words('english')


for personality in ['labels']: #,'cCON','cEXT','cOPN','cNEU']:
    print(personality)
    print('Positive:')
    word_dict = {}
    df_a = df_source[df_source[personality] == 1]
    v = 0
    a = 0
    d = 0

    oov = 0
    total = 0
    for uttr in df_a['utterance']:
        w_list = re.sub(r'[^\w\s\[\]]',' ',uttr.lower()).split()
        for w in w_list:
            if not w in s_list:

                total += 1
                try:
                    v += VAD_dict[w][0]
                    a += VAD_dict[w][1]
                    d += VAD_dict[w][2]

                except:
                    oov += 1
                try:
                    word_dict[w] += 1
                except:
                    word_dict[w] = 1

    match = total-oov
    print(v/match,a/match,d/match)
    print(oov, total)
    
    word_dict = {k: v for k, v in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)}
    
    cnt = 0
    for k,v in word_dict.items():
        print(k, v)
        cnt += 1
        if cnt > 20:
            break
    print('###########################')
    print('Negative:')
    word_dict = {}
    df_a = df_source[df_source[personality] == 0]
    v = 0
    a = 0
    d = 0

    oov = 0
    total = 0
    for uttr in df_a['utterance']:
        w_list = re.sub(r'[^\w\s\[\]]',' ',uttr.lower()).split()
        for w in w_list:
            if not w in s_list:

                total += 1
                try:
                    v += VAD_dict[w][0]
                    a += VAD_dict[w][1]
                    d += VAD_dict[w][2]

                except:
                    oov += 1
                try:
                    word_dict[w] += 1
                except:
                    word_dict[w] = 1

    match = total-oov
    print(v/match,a/match,d/match)
    print(oov, total)
    
    word_dict = {k: v for k, v in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)}
    
    cnt = 0
    for k,v in word_dict.items():
        print(k, v)
        cnt += 1
        if cnt > 20:
            break
    print('###########################')

labels
Positive:
0.6265659190190405 0.4453561033523439 0.5219125450904856
126587 368323
like 5504
really 5446
think 4139
get 3648
know 3597
time 3206
going 3055
go 2936
would 2908
want 2773
people 2659
one 2494
much 2484
feel 2241
well 2152
good 2114
right 1964
need 1911
things 1689
school 1682
wonder 1613
###########################
Negative:
0.6223603618190102 0.4423219830905501 0.5189131074079172
119061 347573
like 5064
really 4870
think 3963
know 3400
get 3274
time 2924
want 2750
go 2714
would 2657
going 2640
people 2459
one 2322
much 2254
well 2093
feel 2021
good 2019
right 1809
need 1711
class 1509
things 1494
school 1475
###########################


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

df_source = pd.read_csv('../../Prompt-Personality/data/Essay/Essay_N_whole.tsv', sep='\t')
df_source.head()

pos = ""
neg = ""
for doc in df_source[df_source['labels'] == 1]['utterance']:
    pos += doc

for doc in df_source[df_source['labels'] == 0]['utterance']:
    neg += doc

    
corpus = [pos, neg]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# print(vectorizer.get_feature_names())
# print(X.shape)


pos_data = { 'word': vectorizer.get_feature_names(),
        'tfidf': X.toarray().tolist()[0]}
df = pd.DataFrame(pos_data)
df = df.sort_values(by="tfidf" , ascending=False) 


n = 200
pos_words = df.head(n)['word']


print('************')
neg_data = { 'word': vectorizer.get_feature_names(),
        'tfidf': X.toarray().tolist()[1]}
df = pd.DataFrame(neg_data)
df = df.sort_values(by="tfidf" , ascending=False) 
neg_words = df.head(n)['word']


print(set(pos_words) - set(neg_words))
print(set(neg_words) - set(pos_words))


************
{'guy', 'everyone', 'keep', 'hate', 'does', 'person', 'sometimes', 'mom'}
{'those', 'your', 'most', 'great', 'best', 'type', 'two', 'read'}
